In [1]:
import os

In [2]:
%pwd

'/mnt/d/DeepLearning/Text_Summarizer_end-to-end_nlp_project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/mnt/d/DeepLearning/Text_Summarizer_end-to-end_nlp_project'

### 1 . Update config.yaml
Done
### 2. Update params.yaml
There is no parameter to update
### 3. Update the entity
Return type of function


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


### 4. Configuration manager in src/config

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


## 5. Update the components

In [8]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging.logger import logging
from text_summarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logging.info(f'{file_name} download! with the following info: \n{headers}')
        else:
            logging.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## 6. Create the pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[ 2023-12-31 02:35:39,929] 36 root - INFO - common - Read yaml file: config/config.yaml
[ 2023-12-31 02:35:39,932] 36 root - INFO - common - Read yaml file: params.yaml
[ 2023-12-31 02:35:39,936] 59 root - INFO - common - Created directory at: artifacts
[ 2023-12-31 02:35:39,941] 59 root - INFO - common - Created directory at: artifacts/data_ingestion
[ 2023-12-31 02:35:39,947] 13 root - INFO - 1747534695 - File already exists of size: ~ 7718 KB
